# Import Modules & Settings

In [1]:
# Import libraries 

import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer

# Import Datasets

In [2]:
# Import dataset

coles_df = pd.read_csv('final_dataset_removed_ALL_assigned_activities.csv')
activities = pd.read_csv('activities.csv')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (23,24,26,27,28,31,32,33,34,36,37,39,40,46,48,49,50,51,55,56,58,59,60,61,62,63,64,67,79,84,86,87,88,89,90,92,93,94,95,96,97,98,99,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Data Exploration

## User Data

In [3]:
coles_df.head()

,actorId,verb,type,activity,activity_url,ancestors,completion,Is Manager,Is New Starter,Time Zone,timestamp,contained,duration
0,0.0,completed,event,activities,https://coles.clearlrs.com/activities/d49baa00...,['activities'],1.0,1.0,0.0,0.0,2021-07-01T00:00:00Z,NaN,0.000
1,0.0,experienced,NaN,learning,https://coles.hub.clearlrs.com/learning,"['home', 'learning']",0.0,1.0,0.0,0.0,2021-07-06T02:08:30Z,NaN,17.738
2,0.0,experienced,NaN,notifications,https://coles.hub.clearlrs.com/profile/notific...,"['home', 'profile', 'notifications']",0.0,1.0,0.0,0.0,2021-07-06T02:08:47.738Z,NaN,0.034
3,0.0,experienced,NaN,profile,https://coles.hub.clearlrs.com/profile,"['home', 'profile']",0.0,1.0,0.0,0.0,2021-07-06T02:08:47.772Z,NaN,25.380
4,0.0,experienced,NaN,explore,https://coles.hub.clearlrs.com/explore,"['home', 'explore']",0.0,1.0,0.0,0.0,2021-07-06T02:09:13.152Z,NaN,14.835


In [4]:
coles_df.shape

(33884834, 13)

In [5]:
coles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33884834 entries, 0 to 33884833
Data columns (total 13 columns):
 #   Column          Dtype  
---  ------          -----  
 0   actorId         float64
 1   verb            object 
 2   type            object 
 3   activity        object 
 4   activity_url    object 
 5   ancestors       object 
 6   completion      float64
 7   Is Manager      float64
 8   Is New Starter  float64
 9   Time Zone       float64
 10  timestamp       object 
 11  contained       object 
 12  duration        float64
dtypes: float64(6), object(7)
memory usage: 3.3+ GB


In [6]:
coles_df.describe()

,actorId,completion,Is Manager,Is New Starter,Time Zone,duration
count,3.388483e+07,3.343349e+07,3.385080e+07,3.388483e+07,3.388483e+07,3.388483e+07
mean,6.578235e+04,3.425496e-01,2.590386e-01,2.815713e-01,2.430423e+00,1.580957e+02
std,4.916339e+04,4.745623e-01,4.381068e-01,4.497654e-01,1.566308e+00,6.719341e+02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-8.515509e+04
25%,2.148200e+04,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.500000e-02
50%,5.764500e+04,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,5.493000e+00
75%,1.046420e+05,1.000000e+00,1.000000e+00,1.000000e+00,4.000000e+00,4.801000e+01
max,1.747330e+05,1.000000e+00,1.000000e+00,1.000000e+00,8.000000e+00,1.079972e+04


In [7]:
coles_df['actorId'].nunique()

174702

In [8]:
coles_df['Is New Starter'].value_counts()

0.0    24343837
1.0     9540997
Name: Is New Starter, dtype: int64

In [9]:
coles_df['verb'].value_counts()

experienced     12057234
completed        9903541
viewed           3466918
searched         3321551
rated            1887255
passed            985334
selected          516498
launched          425524
registered        397083
terminated        246134
removed           225565
suspended         208222
assigned           84776
unregistered       60387
failed             55130
shared             22078
updated            11781
earned              8519
scored               940
attempted            318
reset                 46
Name: verb, dtype: int64

In [10]:
coles_df['type'].value_counts()

lesson             4957425
module             3972310
search-engine      3321473
course             1187114
video               871629
interaction         528357
alert               203939
resource             60643
page                 59898
program              17830
assessment           13846
event                12938
badge                 8519
webpage               1983
event-series           504
session-series         392
voucher                322
article                278
cmi.interaction        183
event-session          105
leaderboard              2
Name: type, dtype: int64

In [11]:
coles_df['activity'].value_counts()

activities             12574367
learning                5782323
results                 3466918
search                  3321551
notifications           1400363
profile                 1262297
others                  1149717
team                     888640
dashboard                765923
tasks                    638797
explore                  535841
course_id                479175
transcript               367118
learningseat_course      307821
brand                    305482
category                 211016
overdue                  192498
due-soon                 111680
colesplay                107751
photo                     11781
events                     3775
Name: activity, dtype: int64

## Activity Data

In [12]:
activities.head()

,Unnamed: 0,id,name,Authoring Tool,Brand,Department,Duration,Functional Owner,Objectives,Passing Score,...,S2S Banner Choice,category,Functional Label,Alternate Products,Cooking Masterclass,Cooking Methods,Cut,Link From Reference,Product Care,Functional OWner
0,0,CEXP:_Advisory,CEXP: Advisory,Storyline,['Express'],All,0.25,Operations,NaN,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,CEXP:_Early_Intervention_Refresher,CEXP: Early Intervention Refresher,Storyline,['Express'],All,0.08,Safety,NaN,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,CEXP:_Emergency_Evacuation_Refresher,CEXP: Emergency Evacuation Refresher,Storyline,['Express'],All,0.08,Safety,NaN,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,CEXP:_Food_Safety_Refresher,CEXP: Food Safety Refresher,Storyline,['Express'],All,0.5,Safety,NaN,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,CEXP:_Forecourt_Safety_Refresher,CEXP: Forecourt Safety Refresher,Storyline,['Express'],All,0.75,Safety,NaN,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
activities_clean = activities[['id', 'CMS Type']]
activities_clean.head()

,id,CMS Type
0,CEXP:_Advisory,elearn
1,CEXP:_Early_Intervention_Refresher,elearn
2,CEXP:_Emergency_Evacuation_Refresher,elearn
3,CEXP:_Food_Safety_Refresher,elearn
4,CEXP:_Forecourt_Safety_Refresher,elearn


# Merge Activity Type to User Data through activity_url

In [14]:
df_new = pd.merge(coles_df, activities_clean, left_on = 'activity_url', right_on = 'id', how = 'left')
df_new.head()

,actorId,verb,type,activity,activity_url,ancestors,completion,Is Manager,Is New Starter,Time Zone,timestamp,contained,duration,id,CMS Type
0,0.0,completed,event,activities,https://coles.clearlrs.com/activities/d49baa00...,['activities'],1.0,1.0,0.0,0.0,2021-07-01T00:00:00Z,NaN,0.000,https://coles.clearlrs.com/activities/d49baa00...,NaN
1,0.0,experienced,NaN,learning,https://coles.hub.clearlrs.com/learning,"['home', 'learning']",0.0,1.0,0.0,0.0,2021-07-06T02:08:30Z,NaN,17.738,NaN,NaN
2,0.0,experienced,NaN,notifications,https://coles.hub.clearlrs.com/profile/notific...,"['home', 'profile', 'notifications']",0.0,1.0,0.0,0.0,2021-07-06T02:08:47.738Z,NaN,0.034,NaN,NaN
3,0.0,experienced,NaN,profile,https://coles.hub.clearlrs.com/profile,"['home', 'profile']",0.0,1.0,0.0,0.0,2021-07-06T02:08:47.772Z,NaN,25.380,NaN,NaN
4,0.0,experienced,NaN,explore,https://coles.hub.clearlrs.com/explore,"['home', 'explore']",0.0,1.0,0.0,0.0,2021-07-06T02:09:13.152Z,NaN,14.835,NaN,NaN


# Split into 2 datasets

In [15]:
df_new_starter = df_new[df_new['Is New Starter'] == 1]
df_non_new_starter = df_new[df_new['Is New Starter'] == 0]

# New Starter Data

In [16]:
df_new_starter = df_new_starter.fillna('')

In [17]:
df_new_starter['combined'] = df_new_starter['verb'] + "|" + df_new_starter['type'] + "|" + df_new_starter['activity'] + "|" + df_new_starter['CMS Type']

In [18]:
# Combined, Verb, Activity, Type, CMS_Type, count, action, social, exploring, learning
## Action - If CMS Type is blank, verb + type, If type is blank, verb + activity, If activity = activities = action - NULL
df_va = df_new_starter[["combined", "verb", "activity", "type", "CMS Type"]]
df_va['count'] = df_va.groupby('combined')['combined'].transform('count')
def action_column(df):
    if df['CMS Type'] == '':
        if df['type'] == '' and df['activity'] == 'activities':
            return 'NULL'
        elif df['type'] == '' and df['activity'] != 'activities':
            return df['verb'] + '_' + df['activity']
        else:
            return df['verb'] + '_' + df['type']
    else:
        return df['verb'] + '_' + df['CMS Type']
df_va['action'] = df_va.apply(lambda df: action_column(df), axis = 1)
df_va

/var/folders/bp/dzdq45bs3sgbprpffw5tfsk40000gn/T/ipykernel_71238/1805242421.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_va['count'] = df_va.groupby('combined')['combined'].transform('count')
/var/folders/bp/dzdq45bs3sgbprpffw5tfsk40000gn/T/ipykernel_71238/1805242421.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_va['action'] = df_va.apply(lambda df: action_column(df), axis = 1)


,combined,verb,activity,type,CMS Type,count,action
20742,registered|lesson|activities|elearn,registered,activities,lesson,elearn,28408,registered_elearn
20743,experienced||learning|,experienced,learning,,,1597158,experienced_learning
20744,unregistered|lesson|activities|elearn,unregistered,activities,lesson,elearn,2069,unregistered_elearn
20745,unregistered|module|activities|Page,unregistered,activities,module,Page,2664,unregistered_Page
20746,unregistered|module|activities|Page,unregistered,activities,module,Page,2664,unregistered_Page
...,...,...,...,...,...,...,...
33884793,completed|module|activities|Page,completed,activities,module,Page,706299,completed_Page
33884794,rated|course|activities|Checklist,rated,activities,course,Checklist,89037,rated_Checklist
33884795,experienced||profile|,experienced,profile,,,285400,experienced_profile
33884796,experienced||notifications|,experienced,notifications,,,324565,experienced_notifications


In [19]:
# Social = True - rated, scored, shared
# Social & Exploring = True - updated_interaction, experienced_profile, experienced_team
# Exploring = True - assigned_Certificate upload,completed_Certificate upload,experienced_dashboard,experienced_events,experienced_explore,experienced_tasks,experienced_transcript,launched_Certificate upload,launched_Checklist,registered_Certificate upload,searched_search-engine,searched_interaction,selected_interaction,viewed_results
# Exploring & Learning = True - assigned_Checklist,assigned_elearn,assigned_event,assigned_Portal Page,assigned_Page,assigned_Video,attempted_elearn,experienced_elearn,experienced_learning,launched_elearn,launched_External Link,launched_module,launched_Page,launched_Video,launched_webpage,passed_elearn,registered_Checklist,registered_elearn,registered_Portal Page,registered_Reward,registered_External Link,registered_Page,registered_session-series,registered_Video,registered_webpage
# Learning = True - completed_Checklist,completed_elearn,completed_External Link,completed_Document,completed_event,completed_event-series,completed_lesson,completed_module,completed_Page,completed_Portal Page,completed_session-series,completed_Video,completed_webpage,earned_badge (earned verb)

def social_column(df):
    if df['verb'] == 'rated' or df['verb'] == 'scored' or df['verb'] == 'shared':
        return True
    elif df['action'] == 'updated_interaction' or df['action'] == 'experienced_profile' or df['action'] == 'experienced_team':
        return True
    else:
        return False

def exploring_column(df):
    if df['action'] == 'updated_interaction' or df['action'] == 'experienced_profile' or df['action'] == 'experienced_team':
        return True
    if df['action'] == 'assigned_Certificate upload' or df['action'] == 'completed_Certificate upload' or df['action'] == 'experienced_dashboard' or df['action'] == 'experienced_events' or df['action'] == 'experienced_explore' or df['action'] == 'experienced_tasks' or df['action'] == 'experienced_transcript' or df['action'] == 'launched_Certificate upload' or df['action'] == 'launched_Checklist' or df['action'] == 'registered_Certificate upload' or df['action'] == 'searched_search-engine' or df['action'] == 'searched_interaction' or df['action'] == 'selected_interaction' or df['action'] == 'viewed_results':
        return True
    if df['action'] == 'assigned_Checklist' or df['action'] == 'assigned_elearn' or df['action'] == 'assigned_event' or df['action'] == 'assigned_Portal Page' or df['action'] == 'assigned_Page' or df['action'] == 'assigned_Video' or df['action'] == 'attempted_elearn' or df['action'] == 'experienced_elearn' or df['action'] == 'experienced_learning' or df['action'] == 'launched_elearn' or df['action'] == 'launched_External Link' or df['action'] == 'launched_module' or df['action'] == 'launched_Page' or df['action'] == 'launched_Video' or df['action'] == 'launched_webpage' or df['action'] == 'passed_elearn' or df['action'] == 'registered_Checklist' or df['action'] == 'registered_elearn' or df['action'] == 'registered_Portal Page' or df['action'] == 'registered_Reward' or df['action'] == 'registered_External Link' or df['action'] == 'registered_Page' or df['action'] == 'registered_session-series' or df['action'] == 'registered_Video' or df['action'] == 'registered_webpage':
        return True
    else:
        return False

def learning_column(df):
    if df['action'] == 'assigned_Checklist' or df['action'] == 'assigned_elearn' or df['action'] == 'assigned_event' or df['action'] == 'assigned_Portal Page' or df['action'] == 'assigned_Page' or df['action'] == 'assigned_Video' or df['action'] == 'attempted_elearn' or df['action'] == 'experienced_elearn' or df['action'] == 'experienced_learning' or df['action'] == 'launched_elearn' or df['action'] == 'launched_External Link' or df['action'] == 'launched_module' or df['action'] == 'launched_Page' or df['action'] == 'launched_Video' or df['action'] == 'launched_webpage' or df['action'] == 'passed_elearn' or df['action'] == 'registered_Checklist' or df['action'] == 'registered_elearn' or df['action'] == 'registered_Portal Page' or df['action'] == 'registered_Reward' or df['action'] == 'registered_External Link' or df['action'] == 'registered_Page' or df['action'] == 'registered_session-series' or df['action'] == 'registered_Video' or df['action'] == 'registered_webpage':
        return True
    if df['verb'] == 'earned' or df['action'] == 'completed_Checklist' or df['action'] == 'completed_elearn' or df['action'] == 'completed_External Link' or df['action'] == 'completed_Document' or df['action'] == 'completed_event' or df['action'] == 'completed_event-series' or df['action'] == 'completed_lesson' or df['action'] == 'completed_module' or df['action'] == 'completed_Page' or df['action'] == 'completed_Portal Page' or df['action'] == 'completed_session-series' or df['action'] == 'completed_Video' or df['action'] == 'completed_webpage':
        return True
    else:
        return False

df_va['social'] = df_va.apply(lambda df: social_column(df), axis = 1)
df_va['exploring'] = df_va.apply(lambda df: exploring_column(df), axis = 1)
df_va['learning'] = df_va.apply(lambda df: learning_column(df), axis = 1)
df_va

/var/folders/bp/dzdq45bs3sgbprpffw5tfsk40000gn/T/ipykernel_71238/872283256.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_va['social'] = df_va.apply(lambda df: social_column(df), axis = 1)
/var/folders/bp/dzdq45bs3sgbprpffw5tfsk40000gn/T/ipykernel_71238/872283256.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_va['exploring'] = df_va.apply(lambda df: exploring_column(df), axis = 1)
/var/folders/bp/dzdq45bs3sgbprpffw5tfsk40000gn/T/ipykernel_71238/872283256.py:35: SettingWithCopyWarning: 
A v

,combined,verb,activity,type,CMS Type,count,action,social,exploring,learning
20742,registered|lesson|activities|elearn,registered,activities,lesson,elearn,28408,registered_elearn,False,True,True
20743,experienced||learning|,experienced,learning,,,1597158,experienced_learning,False,True,True
20744,unregistered|lesson|activities|elearn,unregistered,activities,lesson,elearn,2069,unregistered_elearn,False,False,False
20745,unregistered|module|activities|Page,unregistered,activities,module,Page,2664,unregistered_Page,False,False,False
20746,unregistered|module|activities|Page,unregistered,activities,module,Page,2664,unregistered_Page,False,False,False
...,...,...,...,...,...,...,...,...,...,...
33884793,completed|module|activities|Page,completed,activities,module,Page,706299,completed_Page,False,False,True
33884794,rated|course|activities|Checklist,rated,activities,course,Checklist,89037,rated_Checklist,True,False,False
33884795,experienced||profile|,experienced,profile,,,285400,experienced_profile,True,True,False
33884796,experienced||notifications|,experienced,notifications,,,324565,experienced_notifications,False,False,False


## Merge with verb-activity group

In [20]:
df_va = df_va.rename(columns = {'combined': 'unique_values'})
df_va.head()

,unique_values,verb,activity,type,CMS Type,count,action,social,exploring,learning
20742,registered|lesson|activities|elearn,registered,activities,lesson,elearn,28408,registered_elearn,False,True,True
20743,experienced||learning|,experienced,learning,,,1597158,experienced_learning,False,True,True
20744,unregistered|lesson|activities|elearn,unregistered,activities,lesson,elearn,2069,unregistered_elearn,False,False,False
20745,unregistered|module|activities|Page,unregistered,activities,module,Page,2664,unregistered_Page,False,False,False
20746,unregistered|module|activities|Page,unregistered,activities,module,Page,2664,unregistered_Page,False,False,False


In [21]:
df_new_starter_short = df_new_starter[['actorId', 'combined']]
df_new_starter_short.head()

,actorId,combined
20742,12.0,registered|lesson|activities|elearn
20743,12.0,experienced||learning|
20744,12.0,unregistered|lesson|activities|elearn
20745,12.0,unregistered|module|activities|Page
20746,12.0,unregistered|module|activities|Page


In [13]:
df_va_short = df_va[['unique_values', 'action', 'social', 'exploring', 'learning']]
df_va_short.head()

,unique_values,action,social,exploring,learning
20742,registered|lesson|activities|elearn,registered_elearn,False,True,True
20743,experienced||learning|,experienced_learning,False,True,True
20744,unregistered|lesson|activities|elearn,unregistered_elearn,False,False,False
20745,unregistered|module|activities|Page,unregistered_Page,False,False,False
20746,unregistered|module|activities|Page,unregistered_Page,False,False,False


In [22]:
df_new_starter.to_csv(r'new_starter_data.csv', index = False, header = True)

In [ ]:
df_va.to_csv(r'verb_activity_type_action.csv', index = False, header = True)